In [1]:
import pandas as pd
from google.cloud import bigquery

from etl.utils.utils_methods import load_params

In [2]:
params = load_params()
PROJECT_ID = params["bigquery"]["project_id"]
print("Working project ID: ", PROJECT_ID)

FileNotFoundError: [Errno 2] No such file or directory: 'settings.yaml'

In [ ]:
def run_query(query: str, project_id: str) -> pd.DataFrame:
    """
    Run a query to a specified BigQuery project.

    Args:
        query (str): SQL query to run.
        project_id: ID of the BigQuery project to query.

    Returns:
        (pd.DataFrame): pandas DataFrame with the results of the query.
    """
    client = bigquery.Client(project_id=project_id)
    query_job = client.query(query)
    results = query_job.result()
    return results.to_dataframe()

## Data Overview

## Order trends

## Customer churn

## Customer LTV

## Cohort analysis

## Conclusions